# Import libraries

In [ ]:
from pyspark.sql.functions import expr
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import upper, col, when

# Step 1 - Read historical data using spark dataframe reader

In [ ]:

#Copy the sample code by following the instructions from the manual
customers = spark.read.table("bronze.historical.labelled_customers")


display_cols = ["customerid", "gender", "totalcharges", "churn"]
display(customers.select(display_cols))

# Select the relevant attributes 

In [ ]:
from pyspark.sql import functions as F

# Define the required columns
required_columns = [
    "customerid", 
    "gender", 
    "seniorcitizen", 
    "partner", 
    "dependents", 
    "tenure", 
    "phoneservice", 
    "multiplelines", 
    "internetservice", 
    "onlinesecurity", 
    "onlinebackup", 
    "deviceprotection", 
    "techsupport", 
    "streamingtv", 
    "streamingmovies", 
    "contract", 
    "paperlessbilling", 
    "paymentmethod", 
    "monthlycharges", 
    "totalcharges", 
    "churn", 
    "year", 
    "review"
]

# Create the new DataFrame with the required columns
customers_selected = customers.select(*required_columns)

# Select last 4 years data using Code Assist

### Prompt: Create a dataframe filtered_df from customers_selected with records after 2022

In [ ]:
from pyspark.sql import functions as F

# Filter the DataFrame to include only records after 2022
filtered_df = customers_selected.filter(F.col("year") > 2022)

# Drop rows with blank columns and NAs

In [ ]:
# Import the functions module from pyspark.sql to leverage its built-in functions
from pyspark.sql import functions as F

# Drop rows with missing values from the filtered DataFrame to obtain a cleaned dataset
filtered_customers = filtered_df.dropna()

# Call GenAI model for sentiment analysis of customer reviews

In [ ]:
filtered_customers = filtered_customers.withColumn('sentimentScore_str', expr("query_model('meta.llama-3.2-90b-vision-instruct', concat('What is the sentiment of the review text on a scale of 1 to 5, please give the output as an integer only', review))"))
display(filtered_customers.select('customerid', 'gender', 'monthlycharges', 'totalcharges', 'churn', 'review', 'sentimentScore_str'))

In [ ]:
filtered_customers = filtered_customers.withColumn("sentimentScore", filtered_customers["sentimentScore_str"].cast(IntegerType()))

In [ ]:
filtered_customers = filtered_customers.withColumn(
    "churn",
    when(upper(col("churn")) == "YES", 1)
    .when(upper(col("churn")) == "NO", 0)
)

# Create schema named historical if one does not exist

In [ ]:
spark.sql("CREATE SCHEMA IF NOT EXISTS silver.historical").show()

# Write cleansed customer dataframe to silver layer

In [ ]:
table_name = "silver.historical.customers"
filtered_customers.write.mode("overwrite").format("delta").saveAsTable(table_name)

In [ ]:
df = spark.read.table(table_name)
df.printSchema()